<a href="https://colab.research.google.com/github/Kabongosalomon/Word-Embedding-Investigation/blob/master/%5BTransformer%2C%20BERT%5D%20An%20empirical%20investigation%20into%20the%20properties%20of%20standard%20word%20embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/Kabongosalomon/Data-Science-With-Jeff-Sander/master/aims-za-logo.jpg" width="100" alt="cognitiveclass.ai logo" />





# IMdb Sentiment Analysis Task

## Importing usefull packages 

In [1]:
import torch

import tensorflow as tf
!pip install transformers
import transformers as ppb

import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Downgrate numpy to fix a problem
import numpy as np

import matplotlib.pyplot as plt

import os

import numpy as np

import string

!pip install ipdb
import ipdb # deb

from gensim.models.keyedvectors import KeyedVectors

# Spliting data
from sklearn.model_selection import train_test_split

from sklearn import metrics # For RUC

from nltk.stem import PorterStemmer

import pandas as pd
import re
import seaborn as sns

# from google.colab import files

from IPython import display

import logging
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)

     |████████████████████████████████| 573kB 4.9MB/s 
     |████████████████████████████████| 890kB 15.7MB/s 
     |████████████████████████████████| 1.0MB 27.3MB/s 
     |████████████████████████████████| 3.7MB 42.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=e0203d7ff3aabffcaa93c0392afe3e44a8cbf8e1b9eaf69a8a45810c10cdf7f1
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
  Created wheel for ipdb: filename=ipdb-0.13.2-cp36-none-any.whl size=10522 sha256=abdca7bd6d5d5553640afc18670f66b2e9c1aa31832f36eb596c08a9e29a44b9
  Stored in directory: /root/.cache/pip/wheels/60/c2/15/793365e3c9318c46ba914263740d90f1fe67f544b979141ce4
Success

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
link = "https://drive.google.com/file/d/1smGRs2g2HoI6VSvonoZmWKzXOP6uPUaW/view?usp=sharing"

_, id_t = link.split('d/')

id = id_t.split('/')[0]
print (id) # Verify that you have everything after '='

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


file_id = id
downloaded = drive.CreateFile({'id':file_id})
downloaded.FetchMetadata(fetch_all=True)
downloaded.GetContentFile(downloaded.metadata['title'])

1smGRs2g2HoI6VSvonoZmWKzXOP6uPUaW


In [0]:
!ls

aclImdb.zip  adc.json  sample_data


In [0]:
!unzip -qq aclImdb.zip

In [0]:
!ls

aclImdb  aclImdb.zip  adc.json	__MACOSX  sample_data


In [0]:
# imdb_dir = './data/aclImdb'
imdb_dir = './aclImdb'

# Reading in the training folder
train_dir = os.path.join(imdb_dir, 'train')
texts_tr_ = []
labels_tr = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding="utf8")
            texts_tr_.append(f.read())
            f.close()
            if label_type == 'neg':
                labels_tr.append(0)
            else:
                labels_tr.append(1)

# Reading in the testing folder
train_dir = os.path.join(imdb_dir, 'test')
texts_tst_ = []
labels_tst = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding="utf8")
            texts_tst_.append(f.read())
            f.close()
            if label_type == 'neg':
                labels_tst.append(0)
            else:
                labels_tst.append(1)

In [0]:
# Make sure that we have only 1 and 2 in the label 
(np.unique(labels_tr), np.unique(labels_tst))

(array([0, 1]), array([0, 1]))

In [0]:
len(labels_tr)

25000

In [0]:
# Looking at 2 examples
print(texts_tr_[1])
print(labels_tr[1])

print(texts_tst_[-10])
print(labels_tst[-10])

There are movies that are awful, and there are movies that are so awful they are deemed long-forgotten and unwatchable. Also, lots of violence and bad stuff (not just cheesy stuff; you know what I mean) add to the mix as well. What is the result of bad movies with such raunchy content? Why, "Final Justice," of course! <br /><br />Remember "Mitchell?" Joe Don Baker was the star of that movie, and that was riffed by Joel and the Bots on "Mystery Science Theater 3000." Now this time, with Mike taking Joel's place on the Satellite of Love (but with the same bots), that trio got to make fun of MST3K's second Joe Don Baker movie, "Final Justice." Of course, much of the naughty stuff that I mentioned was removed for television release, but still, I want to watch that episode (and "Mitchell" as well), because what does Joe Don "hate" the most? Why, none other than "Mystery Science Theater 3000!" <br /><br />P.S. If you have a Big Lots nearby, check that store for the uncut tape! LOL That happe

### Getting started

#### Data
We will try to solve the [Large Movie Review Dataset v1.0](http://ai.stanford.edu/~amaas/data/sentiment/) task from Mass et al. The dataset consists of IMDB movie reviews labeled by positivity from 1 to 10. The task is to label the reviews as **negative** or **positive**.

In [2]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
#   data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
#       data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))#; ipdb.set_trace()
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["sentiment"] = 1
  neg_df["sentiment"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df


train_df, test_df = download_and_load_datasets()

84131840/84125825 [==============================] - 1s 0us/step


In [3]:
# Double-check that we've done the right thing.
print("Training examples summary:")
display.display(train_df.describe())
display.display(train_df.head())
print("\n#############################\n")
print("Validation examples summary:")
display.display(test_df.describe())
display.display(test_df.head())

Training examples summary:


,sentiment
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


,sentence,sentiment
0,A great gangster film.Sam Mendes has directed ...,1
1,I MAY have seen an episode or 2 when the show ...,1
2,This movie looked like a classic in the cheesy...,1
3,This is surely one of the worst films ever mad...,0
4,I'm surprised no-one has thought of doing a mo...,1



#############################

Validation examples summary:


,sentiment
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


,sentence,sentiment
0,"Crossfire (1947)<br /><br />Great Message, Gre...",1
1,Great film about an American G.I. who quits th...,1
2,"****SPOILER ALERT**** My boyfriend, some frien...",0
3,The End Of Suburbia (TEOS) is a very useful fi...,1
4,This would have to be by far the greatest seri...,1


In [4]:
train_df['sentence'].apply(lambda x : len(x))

0         691
1         658
2        3106
3         824
4        1404
         ... 
24995     629
24996     315
24997     954
24998     970
24999     665
Name: sentence, Length: 25000, dtype: int64

In [5]:
# mean lengh of review 
train_df['sentence'].apply(lambda x : len(x)).mean()

1325.06964

In [6]:
train_df['sentence'][0]

"A great gangster film.Sam Mendes has directed this beautiful movie showing another father-son camaraderie.Brilliant star-cast leading with Tom Hanks(Michael Sullivan) has done a terrific job.Great acting by him again.He is an acting legend.Great acting too from Paul Newman,Jude Law and Daniel Craig.Casting is just too good.The plot is quite good.You will enjoy the movie.A great portrayal of the gangster of the 1930's.Set in the 1930's,this will surely stand out as the zenith of all gangster movies of that era.Soundtrack is pretty good an apt to the movie.A great flick in totality showing what a father does to protect his son.Way underrated for my liking.Deserved a fully deserved 10."

# Obtain Pre-trained BERT Model

In [0]:
# !pip install torchtext

     |████████████████████████████████| 81kB 158kB/s eta 0:00:01


In [7]:


# # define VGG16 model
# # VGG16 = models.vgg16(pretrained=True)

# DistilBERT = models.transformers()

# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# check if CUDA is available
use_cuda = torch.cuda.is_available()



# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

# move model to GPU if CUDA is available
if use_cuda:
    model = model.cuda()
    # tokenizer = tokenizer.cuda()

In [0]:
class LossPrettifier(object):
    
    STYLE = {
        'green' : '\033[32m',
        'red'   : '\033[91m', 
        'bold'  : '\033[1m', 
    }
    STYLE_END = '\033[0m'
    
    def __init__(self, show_percentage=False):
        
        self.show_percentage = show_percentage
        self.color_up = 'green'
        self.color_down = 'red'
        self.loss_terms = {}
    
    def __call__(self, epoch=None, **kwargs):
        
        if epoch is not None:
            print_string = f'Epoch {epoch: 5d} '
        else:
            print_string = ''

        for key, value in kwargs.items():
            
            pre_value = self.loss_terms.get(key, value)
            
            if value > pre_value:
                indicator  = '▲'
                show_color = self.STYLE[self.color_up]
            elif value == pre_value:
                indicator  = ''
                show_color = ''
            else:
                indicator  = '▼'
                show_color = self.STYLE[self.color_down]
            
            if self.show_percentage:
                show_value = 0 if pre_value == 0 \
                             else (value - pre_value) / float(pre_value)
                key_string = f'| {key}: {show_color}{value:3.4f}({show_value:+3.4%}) {indicator}'
            else: 
                key_string = f'| {key}: {show_color}{value:.4f} {indicator}'
            
            # Trim some long outputs
            key_string_part = key_string[:32]
            print_string += key_string_part+f'{self.STYLE_END}\t'
            
            self.loss_terms[key] = value
            
        print(print_string)
reporter = LossPrettifier(show_percentage=True)

In [0]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss)) 
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss +=((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
            
        # print training/validation statistics 
        
        reporter(epoch=epoch, LossA = train_loss, LossB = valid_loss)
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
    # return trained model
    return model

# loaders_scratch = {'train' : train_loader, 'valid': valid_loader, 'test' :test_loader}

# # train the model
# model_scratch = train(15, loaders_scratch, model_scratch, optimizer_scratch, 
#                       criterion_scratch, use_cuda, 'model_scratch.pt')

In [0]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
# test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

In [11]:
!ls

sample_data


Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

Unfortunatly the bert tokenizer can only encode sequence lenght less or equal to 512

In [0]:
tokenized = train_df['sentence'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512))) 
# tokenized_test = test_df['sentence'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512)))

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" />

### Padding
After tokenization, `tokenized` is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [0]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

# max_len = 0
# for i in tokenized_train.values:
#     if len(i) > max_len:
#         max_len = len(i)

# padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized_train.values])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [14]:
np.array(padded).shape

(25000, 512)

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [15]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(25000, 512)

## Model #1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model!

<img src="http://127.0.0.1:4000/images/distilBERT/bert-distilbert-tutorial-sentence-embedding.png" />

The `model()` function runs our sentences through BERT. The results of the processing will be returned into `last_hidden_states`.

In [17]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

if use_cuda:
    input_ids = input_ids.cuda()
    attention_mask = attention_mask.cuda()


with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


RuntimeError: ignored

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called `[CLS]` (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

<img src="https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png" />

We'll save those in the `features` variable, as they'll serve as the features to our logitics regression model.

In [0]:
features = last_hidden_states[0][:,0,:].numpy()

The labels indicating which sentence is positive and negative now go into the `labels` variable

In [0]:
labels = batch_1[1]

## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 2,000 sentences from the SST2 training set).

<img src="https://jalammar.github.io/images/distilBERT/sentiment-classifier-1.png" />

Under the hood, the model is actually made up of two model.

* DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
* The next model, a basic Logistic Regression model from scikit learn will take in the result of DistilBERT’s processing, and classify the sentence as either positive or negative (1 or 0, respectively).

The data we pass between the two models is a vector of size 768. We can think of this of vector as an embedding for the sentence that we can use for classification.

In [0]:
import os
from torchvision import datasets


## TODO: Specify data loaders
num_workers = 0
# how many samples per batch to load
batch_size = 20


# define training, valid and test data directories
data_dir = './aclImdb/'
train_dir = os.path.join(data_dir, 'train/')
valid_dir = os.path.join(data_dir, 'valid/')
test_dir = os.path.join(data_dir, 'test/')

# data_transform = transforms.Compose([
#     transforms.RandomResizedCrop(224),
#     transforms.RandomHorizontalFlip(), # randomly flip and rotate
# #     transforms.RandomRotation(10),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#     ])

train_data = datasets.ImageFolder(train_dir)
# valid_data = datasets.ImageFolder(valid_dir)
test_data = datasets.ImageFolder(test_dir)


train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers)
# valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, 
#     num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

RuntimeError: ignored

### (IMPLEMENTATION) Model Architecture

Use transfer learning to create a CNN to classify dog breed.  Use the code cell below, and save your initialized model as the variable `model_transfer`.

In [0]:
import torchvision.models as models
import torch.nn as nn

## TODO: Specify model architecture 
# Load the pretrained model from pytorch
model_transfer = models.vgg16(pretrained=True)

# Freeze training for all "features" layers
for param in model_transfer.features.parameters():
    param.requires_grad = False
    
import torch.nn as nn

n_inputs = model_transfer.classifier[6].in_features

# add last linear layer (n_inputs -> 5 flower classes)
# new layers automatically have requires_grad = True
last_layer = nn.Linear(n_inputs, 133)

model_transfer.classifier[6] = last_layer

# check to see that your last layer produces the expected number of outputs
print(model_transfer.classifier[6].out_features)

if use_cuda:
    model_transfer = model_transfer.cuda()

__Question 5:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  Describe why you think the architecture is suitable for the current problem.

__Answer:__ 

- I used the pretrained vgg16 and just changed the last layer to output 133 classes.
- The reason behind using the vgg16 is the it's a pre-trained model on huge images and by freezing the weight we can take advantage of what the model learned from the original training data. 
- I thin kthis architecture is suitable for the dog breed problem because the vgg16 also was trained on imagenet and some of those have some caracteristic in common with my task. 

### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/master/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/master/optim.html).  Save the chosen loss function as `criterion_transfer`, and the optimizer as `optimizer_transfer` below.

In [0]:
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.classifier.parameters(), lr=0.001)

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_transfer.pt'`.

In [0]:
loaders_transfer = {'train' : train_loader, 'valid': valid_loader, 'test' :test_loader}

In [0]:
# train the model
model_transfer = train(15, loaders_transfer, model_transfer, 
                       optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

In [0]:
# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images. Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 60%.

In [0]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

### (IMPLEMENTATION) Predict Dog Breed with the Model

Write a function that takes an image path as input and returns the dog breed (`Affenpinscher`, `Afghan hound`, etc) that is predicted by your model.  

In [0]:
path = '/data/dog_images/train/001.Affenpinscher/Affenpinscher_00001.jpg'
data_transfer = {'train' : train_data, 'valid': valid_data, 'test' :test_data}

In [0]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.

# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in data_transfer['train'].classes]

def predict_breed_transfer(img_path):
    # load the image and return the predicted breed
    
    data_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(), # randomly flip and rotate
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    img = data_transform(Image.open(img_path))
    if use_cuda:
        img = img.cuda()
        
    # PyTorch pretrained models expect the Tensor dims to be (num input imgs, num color channels, height, width).
    # Currently however, we have (num color channels, height, width); let's fix this by inserting a new axis.
    img = img.unsqueeze(0)  # Insert the new axis at index 0 i.e. in front of the other axes/dims. 
    
    prediction = model_transfer(img)  # Returns a Tensor of shape (batch, num class labels)

    
    return class_names[prediction.data.cpu().numpy().argmax()]

---

# <h2>About the Authors:</h2> 

<a href="https://salomonkabongo.wixsite.com/datascientist">Salomon Kabongo KABENAMUALU</a>, Master degree student at <a href="https://aims.ac.za/">the African Institute for mathematical SCiences (AIMS South Africa)</a> his research focused on the use machine learning technique in the field of Natural Language Processing.

References : <a href="https://machinelearningmastery.com/deep-learning-bag-of-words-model-sentiment-analysis/"> How to Develop a Deep Learning Bag-of-Words Model for Predicting Movie Review Sentiment</a>

Copyright &copy; 2019. This notebook and its source code are released under the terms of the <a href="https://www.apache.org/licenses/LICENSE-2.0">Apache License 2.0</a>.